In [3]:
import os
import glob
import cv2
import random
import pandas as pd
from skimage import io
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import h5py

# Network building stuff
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

In [4]:
import sys
sys.path.append("/home2/sdokania/all_projects/project-noisypixel/")

In [6]:
from src.models import *
from src.dataset.dataloader import OccupancyNetDatasetHDF
from src.trainer import ONetLit
from src.utils import Config, count_parameters

In [11]:
vars(config)

{'c_dim': 128,
 'h_dim': 128,
 'p_dim': 3,
 'data_root': '/ssd_scratch/cvit/sdokania/hdf_data/',
 'batch_size': 32,
 'output_dir': '/home2/sdokania/all_projects/occ_artifacts/',
 'exp_name': 'initial',
 'exp_path': '/home2/sdokania/all_projects/occ_artifacts/initial',
 'lr': 0.0003}

In [8]:
config = Config()
config.data_root = "/ssd_scratch/cvit/sdokania/hdf_data/"
config.batch_size = 32

vars(config)

{'c_dim': 128,
 'h_dim': 128,
 'p_dim': 3,
 'data_root': '/ssd_scratch/cvit/sdokania/hdf_data/',
 'batch_size': 32,
 'output_dir': '/home2/sdokania/all_projects/occ_artifacts/',
 'exp_name': 'initial',
 'exp_path': '/home2/sdokania/all_projects/occ_artifacts/initial',
 'lr': 0.0003}

In [9]:
onet = ONetLit(config)

In [6]:
# Initialize tensorboard logger
logger = TensorBoardLogger(
    save_dir=os.getcwd(),
    version=1,
    name="lightning_logs"
)

In [7]:
# Define the trainer object
trainer = pl.Trainer(
    gpus=1,
    # auto_scale_batch_size='binsearch',
    logger=logger,
    min_epochs=1,
    max_epochs=20,
    default_root_dir=config.output_dir,
    log_every_n_steps=10,
    progress_bar_refresh_rate=5,
    # precision=16,
    # stochastic_weight_avg=True,
    track_grad_norm=2,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [7]:
# Start training
trainer.fit(onet)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type      | Params
--------------------------------------------
0 | encoder_model | Resnet50  | 23.8 M
1 | decoder_model | DecoderFC | 182 K 
2 | net           | OccNetImg | 24.0 M
--------------------------------------------
24.0 M    Trainable params
0         Non-trainable params
24.0 M    Total params
95.810    Total estimated model params size (MB)
/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home2/sdokania/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-e977f4c92faa>", line 2, in <module>
    trainer.fit(onet)
  File "/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 499, in fit
    self.dispatch()
  File "/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 546, in dispatch
    self.accelerator.start_training(self)
  File "/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/accelerators/accelerator.py", line 73, in start_training
    self.training_type_plugin.start_training(trainer)
  File "/home2/sdokania/.local/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/training_type_plugin.py", line 114, in start_training
    self._results = trainer.run_train()
  File "/home2/sdokania/.local/li

TypeError: object of type 'NoneType' has no len()